**About** : This notebook is used to ensemble rle files

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import gc
import os
import ast
import sys
import cv2
import glob
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
warnings.simplefilter("ignore", UserWarning)

In [ ]:
from params import *

from utils.plots import *
from utils.metrics import *
from utils.logger import Config
from utils.rle import rle_encode, rle_decode

from inference.tweaking import *
from inference.validation import *
from inference.post_process import *

from data.preparation import prepare_data

## Exps

In [ ]:
df = prepare_data(False)
df = df.sort_values('id').reset_index(drop=True)
df = df[['id', 'annotation', 'cell_type', 'img_path', 'ann']]

In [ ]:
EXP_FOLDER = LOG_PATH + "2021-11-16/3/"

df_1 = pd.read_csv(EXP_FOLDER + "df_oof.csv")
df_1 = df_1.sort_values('id').reset_index(drop=True)

df_1['rles'] = df_1['rles'].apply(ast.literal_eval)
df_1['boxes'] = df_1['boxes'].apply(ast.literal_eval)

In [ ]:
# EXP_FOLDER = LOG_PATH + "2021-11-22/5/"

# df_2 = pd.read_csv(EXP_FOLDER + "df_oof.csv")
# df_2 = df_2.sort_values('id').reset_index(drop=True)

# df_2['rles'] = df_2['rles'].apply(ast.literal_eval)
# df_2['boxes'] = df_2['boxes'].apply(ast.literal_eval)

In [ ]:
# stardist
df_2 = pd.read_csv(OUT_PATH + "preds_stardist_22_11.csv")

df_2 = df_2[
    ['id', 'predicted_rles', 'probabilities', 'scores']
].rename(columns={'scores': 'score', 'predicted_rles': 'rles', 'probabilities': 'boxes'})

df_2['boxes'] = df_2['boxes'].apply(ast.literal_eval)
df_2['boxes'] = df_2['boxes'].apply(lambda x: np.tile(np.array(x)[None], (6, 1)).T)
df_2['rles'] = df_2['rles'].apply(ast.literal_eval)
# 

In [ ]:
df = df.merge(
    df_1, how='left', on="id", suffixes=('', '_1')
).rename(columns={'rles': 'rles_1', 'boxes': 'boxes_1', 'score': 'score_1'})

In [ ]:
df = df.merge(
    df_2, how='left', on="id", suffixes=('', '_2')
).rename(columns={'rles': 'rles_2', 'boxes': 'boxes_2', 'score': 'score_2'})

In [ ]:
# dfg = df[['cell_type', 'score_stardist', 'score_rcnn']].groupby('cell_type').agg(list)

# plt.figure(figsize=(10, 10))

# i = 1
# for col in dfg.columns:
#     for cell_type in dfg.index:
#         scores = dfg[col][cell_type]
#         plt.subplot(len(dfg.columns), len(dfg.index), i)
#         plt.title(str(col + ' - ' + cell_type), size=15)
#         plt.grid(True)
#         plt.scatter(range(len(scores)), sorted(scores), s=20)
#     #     plt.ylim(0, 0.75)
#         plt.xlabel('')
#         plt.ylabel('IoU mAP')
#         i+=1

## Viz

In [ ]:
pipelines = define_pipelines("data/config.py")

dataset = SartoriusDataset(df, transforms=pipelines['val_viz'], precompute_masks=False)

In [ ]:
def score_merge_preds(pred_1, pred_2, truth):
    
    rles_2 = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in pred_2.astype(np.uint8)]
    rles_1 = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in pred_1.astype(np.uint8)]

    ious = pycocotools.mask.iou(rles_1, rles_2, [0] * 100000)
    ious_sum = ious.sum(0)
    
    all_rles = rles_1 + [p for i, p in enumerate(rles_2) if ious_sum[i] <= 0.]
    
    iou = pycocotools.mask.iou(truth, all_rles, [0] * 100000)
    score = iou_map(ious=[iou])

    return score

### Scores

In [ ]:
# scores_1, scores_per_class_1 = evaluate(masks_1, dataset, df['cell_type'].apply(lambda x: CELL_TYPES.index(x)).values)

In [ ]:
# score = np.mean(np.concatenate(scores_per_class_1))
# scores_class = [np.mean(s) if len(s) else 0 for s in scores_per_class_1]

# print(f' -> IoU mAP : {score:.4f}\n')

# for s, c in zip(scores_class, CELL_TYPES):
#     print(f'{c} : {s:.4f}')

In [ ]:
# scores_2, scores_per_class_2 = evaluate(masks_2, dataset, df['cell_type'].apply(lambda x: CELL_TYPES.index(x)).values)

In [ ]:
# score = np.mean(np.concatenate(scores_per_class_2))
# scores_class = [np.mean(s) if len(s) else 0 for s in scores_per_class_2]

# print(f' -> IoU mAP : {score:.4f}\n')

# for s, c in zip(scores_class, CELL_TYPES):
#     print(f'{c} : {s:.4f}')

In [ ]:
# print(f' -> IoU mAP : {np.mean(scores_merge):.4f}\n')

# for idx in range(3):
#     s = np.mean([scores_merge[i] for i, c in enumerate(df['cell_type']) if c == CELL_TYPES[idx]])
#     print(f'{CELL_TYPES[idx]} : {s:.4f}')

### Scores

In [ ]:
masks_merged = []

for i in tqdm(range(len(dataset))):
    # 1
    rles_1 = df['rles_1'][i]
    pred_1 = np.array([rle_decode(enc, ORIG_SIZE) for enc in rles_1])
    
    # 2
    rles_2 = df['rles_2'][i]
    pred_2 = np.array([rle_decode(enc, ORIG_SIZE) for enc in rles_2])
    
    mask_to_merge = np.concatenate([pred_1, pred_2], 0)
    
    b2 = np.zeros(df_2['boxes'][i].shape)
    boxes_to_merge = np.concatenate([df_1['boxes'][i], b2], 0)
    
    masks, boxes, picks = mask_nms(mask_to_merge, boxes_to_merge, 0.00000001)
    
    masks = remove_overlap_naive(masks)
    
    masks_merged.append(masks)    
    
    break

In [ ]:
scores_m, scores_per_class_m = evaluate(masks_merged, dataset, df['cell_type'].apply(lambda x: CELL_TYPES.index(x)).values)

In [ ]:
score = np.mean(np.concatenate(scores_per_class_m))
scores_class = [np.mean(s) if len(s) else 0 for s in scores_per_class_m]

print(f' -> IoU mAP : {score:.4f}\n')

for s, c in zip(scores_class, CELL_TYPES):
    print(f'{c} : {s:.4f}')

In [ ]:
df['score_1'][i], df['score_2'][i]

### Viz

In [ ]:
for idx in tqdm(range(len(dataset))):
    score_1 = df['score_1'][idx]
    cell_1 = df['cell_type_1'][idx]

    score_2 = df['score_2'][idx]
    try:
        cell_2 = df['cell_type_2'][idx]
    except:
        cell_2 = 0
    
    # 1
    rles_1 = df['rles_1'][idx]
    pred_1 = np.array([rle_decode(enc, ORIG_SIZE) for enc in rles_1]).astype(int)
    
    # 2
    rles_2 = df['rles_2'][idx]
    pred_2 = np.array([rle_decode(enc, ORIG_SIZE) for enc in rles_2]).astype(int)
    
    # merged
    mask_m = masks_merged[idx].astype(int).copy()
    
    # plot
    data = dataset[idx]
    img = data['img']
    truth = data['gt_masks'].masks.copy().astype(int)
    
    plt.figure(figsize=(15, 15))
    plot_sample(img, pred_1.copy())
    plt.axis(False)
    plt.title(f'Pred 1 - {CELL_TYPES[cell_1]} - iou_map={score_1:.3f}')
    plt.show()

    plt.figure(figsize=(15, 15))
    plot_sample(img, pred_2.copy())
    plt.axis(False)
    plt.title(f'Pred 2 - {CELL_TYPES[cell_2]} - iou_map={score_2:.3f}')
    plt.show()
    
    plt.figure(figsize=(15, 15))
    plot_sample(img, mask_m)
    plt.axis(False)
    plt.title(f'Pred merged - {CELL_TYPES[cell_1]} - iou_map={0:.3f}')
    plt.show()
    
    plt.figure(figsize=(15, 15))
    plot_sample(img, truth.copy(), plotly=False)
    plt.axis(False)
    plt.title(f'Truth - {df["cell_type"][idx]}')
    plt.show()

    break

In [ ]:
fig = plot_preds_iou(img, pred_1, mask_m, plot_tp=False)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()

In [ ]:
fig = plot_preds_iou(img, mask_m, truth, plot_tp=False)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()

In [ ]:
fig = plot_preds_iou(img, pred_2, mask_m, plot_tp=False)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()

In [ ]:
fig = plot_preds_iou(img, mask_m, truth, plot_tp=False)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()